In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import sys
from typing import Optional
if '..' not in sys.path: sys.path.append('..')

from datasets import load_dataset
from datasets.arrow_dataset import Dataset
import numpy as np
from matplotlib import pyplot as plt
from pydantic_yaml import parse_yaml_file_as
import torch
from torch import nn
from transformers import GPT2Tokenizer, AddedToken, PreTrainedTokenizer

from mllm.data.wiki.dswiki import WikiDsLoader
from mllm.exp.args import TOKENIZER_CFG_FNAME, ENCDEC_HG_MODEL_CFG_FNAME
from mllm.model.encdec_hg import EncdecHg
from mllm.config.model import TokenizerCfg, EncdecHgCfg
from mllm.tokenization.chunk_tokenizer import tokenizer_from_config


/home/misha/miniconda3/envs/mllm/lib/python3.10/site-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/misha/miniconda3/envs/mllm/lib/python3.10/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'
WIKI_DS_NAME = '20200501.en'

TRAIN_ENCDEC_HG_PATH = DATA_PATH / 'train_mllm_encdec_hg'
encdec_subdir = 'encdechg-20241202_225258-ilen256-lrs8-step2-d256-h8'

encdec_train_path = TRAIN_ENCDEC_HG_PATH / encdec_subdir
encdec_snapshot_fpath = encdec_train_path / 'best.pth'
encdec_model_cfg_fpath = encdec_train_path / ENCDEC_HG_MODEL_CFG_FNAME
encdec_tkz_cfg_fpath = encdec_train_path / TOKENIZER_CFG_FNAME

device_name = 'cpu'
# device_name = 'cuda'

device = torch.device(device_name)
print(device)

cpu


In [4]:
dss = load_dataset('wikipedia', WIKI_DS_NAME, beam_runner='DirectRunner', cache_dir=str(DATA_PATH))
ds: Dataset = dss['train']
n_docs = len(ds)
print(f'Wikipedia {WIKI_DS_NAME} docs: {n_docs}')

Reusing dataset wikipedia (/home/misha/data/wikipedia/20200501.en/1.0.0/009f923d9b6dd00c00c8cdc7f408f2b47f45dd4f5fb7982a21f9448f4afbe475)


  0%|          | 0/1 [00:00<?, ?it/s]

Wikipedia 20200501.en docs: 6078422


In [8]:
tkz_cfg = parse_yaml_file_as(TokenizerCfg, encdec_tkz_cfg_fpath)
tkz = tokenizer_from_config(tkz_cfg)
model_cfg = parse_yaml_file_as(EncdecHgCfg, encdec_model_cfg_fpath)
inp_len = model_cfg.enc_pyr.inp_len
pad_tok = tkz_cfg.custom_tokens['pad'].ind

In [7]:
chkpt = torch.load(encdec_snapshot_fpath, map_location=device)
model = EncdecHg(model_cfg).to(device)
strict = True
# strict = False
model.load_state_dict(chkpt['model'], strict=strict)

<All keys matched successfully>

In [15]:
def get_batch_tokens(doc_inds: list[int], randomize: bool = False) -> torch.Tensor:
    docs_toks = np.full((len(doc_inds), inp_len), pad_tok)
    for i, doc_ind in enumerate(doc_inds):
        doc = ds[doc_ind]
        title, text = doc['title'], doc['text']
        doc_txt = f'{title} {text}'
        doc_toks: list[int] = tkz(doc_txt)['input_ids']
        n_toks = len(doc_toks)
        if n_toks > inp_len:
            i_off = np.random.randint(n_toks - inp_len + 1) if randomize else 0
            doc_toks = doc_toks[i_off:i_off + inp_len]
        docs_toks[i, :len(doc_toks)] = doc_toks
    docs_toks_t = torch.from_numpy(docs_toks).to(device)
    return docs_toks_t


In [13]:
doc_inds = np.arange(5)
doc_inds = [x.item() for x in doc_inds]
for doc_ind in doc_inds:
    doc = ds[doc_ind]
    title, text = doc['title'], doc['text'].replace('\n', '\\n')
    print(f'{doc_ind:03d} "{title}" {text[:300]}')

000 "Yangliuqing" Yangliuqing () is a market town in Xiqing District, in the western suburbs of Tianjin, People's Republic of China. Despite its relatively small size, it has been named since 2006 in the "famous historical and cultural market towns in China".\n\nIt is best known in China for creating nianhua or Yangl
001 "Orana Australia Ltd" Orana Australia Ltd is a not-for-profit organisation that provides a diverse range of training and support services to over 650 people with disabilities and their families in South Australia.\n\nHistory\nThe Mentally Retarded Children’s Society of SA Inc. was established in 1950 by a group of parent
002 "St. Mary's Church, Sønderborg" The St. Mary's Church is a church owned by the Church of Denmark in Sønderborg, Denmark and the church of the parish with the same name. Thanks to its location on a hill, the church building is very iconic for the city.\n\nHistory \nIn the Middle Ages there was a leper colony on a hill just outside 
003 "Kalitta" Kal

In [25]:
docs_toks_in = get_batch_tokens(doc_inds)
logits_pred = model(docs_toks_in)
probs_pred = torch.softmax(logits_pred, dim=-1)
# probs_pred = torch.sigmoid(logits_pred)
print(probs_pred.shape)
docs_toks_out = torch.argmax(probs_pred, dim=-1)
print(docs_toks_out.shape)

torch.Size([5, 256, 50271])
torch.Size([5, 256])


In [26]:
for i, doc_ind in enumerate(doc_inds):
    s = tkz.decode(docs_toks_out[i])
    s = s.replace('\n', '\\n')
    print(f'{doc_ind:03d} {s}')


000  ( ( ( ( ( ( ( ( ( () is a the of of ( (,.) is the the the the of of of, of. in century. The is the the the the the the,,,,,,, the the the the the the the..\n\nThe is is the the the the the the ( ( ( ( ( ( ( ( ( (i. The is the the the the the the,,,,,,, the the the the the the the the the the the the., the the the the the the..,, the the the the the the the the the the the the the the the the the..\n\nThe is the the the the the the the the the the the.\n\nThe:\n of of:\n\nThe:\n ( ( ( (\n\n\n�� ( ( ( ( ( (������������������\n) is the the the ( ( ( ( ( ( ( ( (.. The the the the the the the of. of.., the the the the the the.., the the the the the the the the the of., the the the
001  ( ( ( ( ( (.) is a a the the the,,, the the the the the the, and the the the the the the the the the the the the..\n\nHistory\n\n\n\n\n\n,,,,,,,., and the the the the the,,, the the, the the the the the the the the the the,,,,,,,, the the the to the the time.\n\nIn the,,,, is, the the the the the the the